In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.8 MB/s eta 0:00:00


In [9]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
device

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 4090 Laptop GPU


device(type='cuda')

## Модель

Загружаем [ruGPT3](https://huggingface.co/ai-forever/rugpt3large_based_on_gpt2).

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

In [11]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1536, out_features=50257, bias=False)
)

# Zero-shot

Для классификации мы будем использовать loss модели.

**Идея:** посмотреть, для какого варианта ответа loss моделибудет меньше и выбрать его.


In [12]:
import math
def calc_loss(phrase: str,
                        tokenizer,
                        model):

    phrase = tokenizer.encode(phrase)
    # Если длина фразы 1 токен, то дальше ошибка вылезет :(
    if len(phrase) == 1:
         phrase.append(tokenizer.eos_token_id)
    phrase = torch.tensor(phrase, dtype=torch.long, device=device)
    phrase = phrase.unsqueeze(0)  # .repeat(num_samples, 1)
    with torch.no_grad():
        loss = model(phrase, labels=phrase)

    loss[0].item()


    return loss[0].item()

def get_loss_num(text):
    loss = calc_loss(phrase=text, model=model, tokenizer=tokenizer)
    return loss

def clean(text):
    text = re.sub(r'\((\d+)\)', '', text)
    return text

### Анализ тональности (твиттер)

In [13]:
text = 'жизнь отличная'
get_loss_num('Позитивный твит: ' + text + ')))')

6.151875972747803

In [14]:
get_loss_num('Негативный твит: ' + text + '(((')

7.050113677978516

### Анализ грамматической коррекности

In [16]:
text = 'Кот сидел на коврике.'
few_shots = ['Предложение далее корректное? ' + 'Мама мыла раму.' + " Ответ: да.",
             'Предложение далее корректное? ' + 'Пингвить идить туды.' + " Ответ: нет."]
correct_loss = get_loss_num('\n'.join(few_shots) +'\nПредложение далее корректное? ' + text + " Ответ: да.")
correct_loss

3.07968807220459

In [17]:
text = 'Кот сидеть на коврике.'
incorrect_loss = get_loss_num('\n'.join(few_shots) +'Предложение далее корректное? ' + text + " Ответ: нет.")
incorrect_loss

3.4581687450408936